In [9]:
%pip install --upgrade --quiet  \
    langchain-openai \
    langchain \
    langchain-community \
    langchain-google-genai\
    langchain-groq\
    langchain-experimental\
    networkx\
    langchain-chroma\
    json-repair\
    langchain-openai\
    neo4j


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# https://medium.com/@sridevi.gogusetty/rag-vs-graph-rag-llama-3-1-8f2717c554e6

In [12]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_google_genai import GoogleGenerativeAI
from langchain.indexes.graph import NetworkxEntityGraph
from langchain.chains import GraphQAChain
from langchain_groq import ChatGroq

GROQ_API_KEY = ""

In [13]:
Text = 'Rani Lakshmibai (or Rani Lakshmi Bai) was born on 19 November 1828, (some sources say 1835) in the town of Banares (now Varanasi) into a Marathi Karhade Brahmin family.\
 She was named Manikarnika Tambe and was nicknamed Manu.[8] Her father was Moropant Tambe[9] and her mother Bhagirathi Sapre (Bhagirathi Bai).\
  Her parents came from the Tambe village of the Guhagar taluka located in the Ratnagiri district of modern-day Maharashtra.\
 Her mother died when she was five years old. Her father was a Commander during the war of Kalyanpranth. Her father worked for Peshwa Baji Rao II of Bithoor district.\
  The Peshwa called her "Chhabili", which means "beautiful " and "lively and cheerful". \
 She was educated at home and was taught to read and write, and was more independent in her childhood than others of her age; \
 her studies included shooting, horsemanship, fencing and mallakhamba with her childhood friends Nana Sahib and Tantia Tope. \
 Rani Lakshmibai contrasted many of the patriarchal cultural expectations for women in India\'s society at this time.\
And she was known for her unique perspectives and her courage to fight against social norms even in front of the whole society.\
Rani Lakshmibai was accustomed to riding on horseback accompanied by escorts between the palace and the temple, although sometimes she was carried in a palanquin.\
Her horses included Sarangi, Pavan and Baadal; according to historians, she rode Baadal when escaping from the fort in 1858. Her palace, the Rani Mahal, has now been converted into a museum.\
 It houses a collection of archaeological remains of the period between the 9th and 12th centuries AD.'
documents = [Document(page_content=Text)]
llm = ChatGroq(
    temperature=0, model_name="llama-3.1-70b-versatile", groq_api_key=GROQ_API_KEY
)
llm_transformer_filtered = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Person", "Country", "Organization"],
    allowed_relationships=[
        "NATIONALITY",
        "Father",
        "Mother",
        "LOCATED_IN",
        "WORKED_AT",
        "SPOUSE",
    ],
)
graph_documents_filtered = llm_transformer_filtered.convert_to_graph_documents(
    documents
)
graph = NetworkxEntityGraph()
# add nodes to the graph
for node in graph_documents_filtered[0].nodes:
    graph.add_node(node.id)
# add edges to the graph
for edge in graph_documents_filtered[0].relationships:
    graph._graph.add_edge(edge.source.id, edge.target.id, relation=edge.type)
print(f"Nodes:{graph_documents_filtered[0].nodes}")
print(f"Relationships:{graph_documents_filtered[0].relationships}")
graph_rag = GraphQAChain.from_llm(llm, graph=graph, verbose=True)

Nodes:[Node(id='Rani Lakshmibai', type='Person', properties={}), Node(id='Moropant Tambe', type='Person', properties={}), Node(id='Bhagirathi Sapre', type='Person', properties={}), Node(id='Peshwa Baji Rao Ii', type='Person', properties={}), Node(id='Nana Sahib', type='Person', properties={}), Node(id='Tantia Tope', type='Person', properties={}), Node(id='Banares', type='Country', properties={}), Node(id='Varanasi', type='Country', properties={}), Node(id='Tambe Village', type='Country', properties={}), Node(id='Guhagar Taluka', type='Country', properties={}), Node(id='Ratnagiri District', type='Country', properties={}), Node(id='Maharashtra', type='Country', properties={}), Node(id='Bithoor District', type='Country', properties={}), Node(id='Rani Mahal', type='Organization', properties={})]
Relationships:[Relationship(source=Node(id='Rani Lakshmibai', type='Person', properties={}), target=Node(id='Moropant Tambe', type='Person', properties={}), type='FATHER', properties={}), Relations

In [15]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA

In [19]:
chain = GraphQAChain.from_llm(llm=llm, graph=graph, verbose=True)
question = """Rani Lakshmibai father"""
chain.run(question)



> Entering new GraphQAChain chain...
Entities Extracted:
Rani Lakshmibai
Full Context:
Rani Lakshmibai FATHER Moropant Tambe
Rani Lakshmibai MOTHER Bhagirathi Sapre
Rani Lakshmibai WORKED_AT Peshwa Baji Rao Ii
Rani Lakshmibai SPOUSE Nana Sahib
Rani Lakshmibai SPOUSE Tantia Tope
Rani Lakshmibai LOCATED_IN Banares
Rani Lakshmibai LOCATED_IN Varanasi
Rani Lakshmibai LOCATED_IN Tambe Village
Rani Lakshmibai LOCATED_IN Guhagar Taluka
Rani Lakshmibai LOCATED_IN Ratnagiri District
Rani Lakshmibai LOCATED_IN Maharashtra
Rani Lakshmibai LOCATED_IN Bithoor District
Rani Lakshmibai WORKED_AT Rani Mahal

> Finished chain.


"Rani Lakshmibai's father is Moropant Tambe."